# Preprocessing dan Ekstraksi Fitur TF-IDF

---

### Konsep yang Digunakan:
- **TF (Term Frequency)**: `TF(t,d) = jumlah kemunculan kata t / total kata dalam dokumen d`
- **IDF (Inverse Document Frequency)**: `IDF(t) = log(N / df(t))`
- **TF-IDF**: `TF-IDF(t,d) = TF(t,d) × IDF(t)`

## 1. Fungsi Helper (Tanpa Library)

In [ ]:
def hitung_log(x):
    """
    Menghitung logaritma natural menggunakan ekspansi Taylor Series
    ln(x) = (x-1) - (x-1)^2/2 + (x-1)^3/3 - ...
    """
    if x <= 0:
        return float('-inf')

    # Normalisasi x ke range [0.5, 1.5] untuk akurasi lebih baik
    exp_adjust = 0
    while x > 1.5:
        x = x / 2.718281828459045  # e
        exp_adjust += 1
    while x < 0.5:
        x = x * 2.718281828459045
        exp_adjust -= 1

    # Ekspansi Taylor
    result = 0
    z = x - 1
    term = z

    for n in range(1, 100):
        result += term / n
        term *= -z
        if abs(term / n) < 0.0000001:  # Konvergensi
            break

    return result + exp_adjust


def adalah_huruf(char):
    """Cek apakah karakter adalah huruf"""
    return ('a' <= char <= 'z') or ('A' <= char <= 'Z')


def adalah_angka(char):
    """Cek apakah karakter adalah angka"""
    return '0' <= char <= '9'


def adalah_whitespace(char):
    """Cek apakah karakter adalah whitespace"""
    return char in ' \t\n\r'


def ke_huruf_kecil(char):
    """Ubah karakter ke huruf kecil"""
    if 'A' <= char <= 'Z':
        return chr(ord(char) + 32)
    return char


def cetak_garis(panjang=70, karakter="="):
    """Mencetak garis pemisah"""
    for i in range(panjang):
        print(karakter, end="")
    print()


def format_angka(angka, desimal=4):
    """Format angka dengan jumlah desimal tertentu"""
    format_str = "{:." + str(desimal) + "f}"
    return format_str.format(angka)

## 2. Class TextPreprocessor

Class ini melakukan preprocessing teks dengan tahapan:
1. Lowercase (ubah ke huruf kecil)
2. Remove punctuation (hapus tanda baca)
3. Remove numbers (hapus angka)
4. Tokenization (pecah menjadi kata)
5. Remove stopwords (hapus kata-kata umum)

In [ ]:
class TextPreprocessor:
    """Kelas untuk preprocessing teks"""

    def __init__(self):
        # Daftar stopwords bahasa Indonesia
        self.stopwords = [
            'yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia',
            'dua', 'ia', 'seperti', 'jika', 'sehingga', 'kembali', 'dan', 'ini',
            'itu', 'dalam', 'tidak', 'adalah', 'sebagai', 'lebih', 'akan', 'telah',
            'dari', 'oleh', 'di', 'dengan', 'atau', 'ada', 'juga', 'bahwa', 'tersebut',
            'sangat', 'dapat', 'sudah', 'bisa', 'karena', 'hanya', 'masih', 'yaitu'
        ]

    def lowercase(self, text):
        """Mengubah teks menjadi huruf kecil"""
        result = ""
        for char in text:
            result += ke_huruf_kecil(char)
        return result

    def remove_punctuation_and_numbers(self, text):
        """Menghapus tanda baca dan angka"""
        result = ""
        for char in text:
            if adalah_huruf(char) or adalah_whitespace(char):
                result += char
            elif not adalah_angka(char):
                result += ' '  # Ganti tanda baca dengan spasi
        return result

    def tokenize(self, text):
        """Memecah teks menjadi token/kata"""
        tokens = []
        current_token = ""

        for char in text:
            if adalah_whitespace(char):
                if current_token:
                    tokens.append(current_token)
                    current_token = ""
            else:
                current_token += char

        # Tambahkan token terakhir
        if current_token:
            tokens.append(current_token)

        return tokens

    def remove_stopwords(self, tokens):
        """Menghapus stopwords"""
        result = []
        for token in tokens:
            # Cek panjang token dan apakah stopword
            is_stopword = False
            for sw in self.stopwords:
                if token == sw:
                    is_stopword = True
                    break

            if not is_stopword and len(token) > 2:
                result.append(token)

        return result

    def preprocess(self, text):
        """Pipeline lengkap preprocessing"""
        text = self.lowercase(text)
        text = self.remove_punctuation_and_numbers(text)
        tokens = self.tokenize(text)
        tokens = self.remove_stopwords(tokens)
        return tokens


print("✓ Class TextPreprocessor berhasil didefinisikan!")
print("\nContoh penggunaan:")
preprocessor = TextPreprocessor()
contoh_teks = "Python adalah bahasa pemrograman yang populer!"
hasil = preprocessor.preprocess(contoh_teks)
print(f"  Input : {contoh_teks}")
print(f"  Output: {hasil}")

✓ Class TextPreprocessor berhasil didefinisikan!

Contoh penggunaan:
  Input : Python adalah bahasa pemrograman yang populer!
  Output: ['python', 'bahasa', 'pemrograman', 'populer']


## 3. Class TFIDFExtractor

Class ini menghitung TF-IDF untuk setiap dokumen dengan langkah:
1. **Fit**: Membangun vocabulary dan menghitung IDF
2. **Transform**: Mengubah dokumen menjadi vektor TF-IDF

In [ ]:
class TFIDFExtractor:
    """Kelas untuk menghitung TF-IDF tanpa library"""

    def __init__(self):
        self.vocabulary = []
        self.idf_values = {}
        self.documents = []

    def hitung_kata(self, tokens):
        """Menghitung frekuensi setiap kata dalam list tokens"""
        count = {}
        for token in tokens:
            if token in count:
                count[token] += 1
            else:
                count[token] = 1
        return count

    def kata_unik(self, tokens):
        """Mendapatkan kata-kata unik dari list tokens"""
        unique = []
        for token in tokens:
            if token not in unique:
                unique.append(token)
        return unique

    def urutkan_list(self, items):
        """Mengurutkan list menggunakan bubble sort"""
        n = len(items)
        for i in range(n):
            for j in range(0, n-i-1):
                if items[j] > items[j+1]:
                    items[j], items[j+1] = items[j+1], items[j]
        return items

    def fit(self, documents):
        """Menghitung IDF dari koleksi dokumen"""
        self.documents = documents
        N = len(documents)  # Jumlah dokumen

        # Bangun vocabulary
        all_words = []
        for doc in documents:
            for word in doc:
                if word not in all_words:
                    all_words.append(word)

        self.vocabulary = self.urutkan_list(all_words)

        # Hitung document frequency untuk setiap kata
        df = {}
        for word in self.vocabulary:
            df[word] = 0

        for doc in documents:
            unique_words = self.kata_unik(doc)
            for word in unique_words:
                if word in df:
                    df[word] += 1

        # Hitung IDF
        for word in self.vocabulary:
            # IDF = log(N / df(t))
            self.idf_values[word] = hitung_log(N / df[word])

        return self

    def calculate_tf(self, tokens):
        """Menghitung Term Frequency (TF)"""
        tf_count = self.hitung_kata(tokens)
        total_words = len(tokens)

        # Normalisasi TF
        tf = {}
        if total_words > 0:
            for word in tf_count:
                tf[word] = tf_count[word] / total_words

        return tf

    def transform(self, documents):
        """Mengubah dokumen menjadi vektor TF-IDF"""
        tfidf_vectors = []

        for doc in documents:
            # Hitung TF
            tf = self.calculate_tf(doc)

            # Hitung TF-IDF
            tfidf = {}
            for word in self.vocabulary:
                if word in tf:
                    tfidf[word] = tf[word] * self.idf_values[word]
                else:
                    tfidf[word] = 0.0

            tfidf_vectors.append(tfidf)

        return tfidf_vectors

    def fit_transform(self, documents):
        """Fit dan transform dalam satu langkah"""
        self.fit(documents)
        return self.transform(documents)

    def get_feature_names(self):
        """Mendapatkan nama fitur (vocabulary)"""
        return self.vocabulary

    def vector_to_list(self, tfidf_vector):
        """Mengubah TF-IDF vector (dict) menjadi list"""
        result = []
        for word in self.vocabulary:
            result.append(tfidf_vector[word])
        return result

    def urutkan_tfidf(self, tfidf_vector):
        """Mengurutkan kata berdasarkan nilai TF-IDF (descending)"""
        items = []
        for word in tfidf_vector:
            items.append((word, tfidf_vector[word]))

        # Bubble sort descending berdasarkan nilai
        n = len(items)
        for i in range(n):
            for j in range(0, n-i-1):
                if items[j][1] < items[j+1][1]:
                    items[j], items[j+1] = items[j+1], items[j]

        return items

## 4. Persiapan Data

Siapkan dokumen teks yang akan diproses

In [ ]:
documents_text = [
    "Python adalah bahasa pemrograman yang populer untuk machine learning",
    "Machine learning membutuhkan data yang banyak untuk pelatihan model",
    "Bahasa pemrograman Python mudah dipelajari oleh pemula",
    "Data science menggunakan Python dan R sebagai bahasa pemrograman utama",
    "Model machine learning dapat digunakan untuk prediksi data"
]

print(f"\nJumlah dokumen: {len(documents_text)}")
print("\nDokumen-dokumen:")
for i, doc in enumerate(documents_text, 1):
    print(f"  {i}. {doc}")


Jumlah dokumen: 5

Dokumen-dokumen:
  1. Python adalah bahasa pemrograman yang populer untuk machine learning
  2. Machine learning membutuhkan data yang banyak untuk pelatihan model
  3. Bahasa pemrograman Python mudah dipelajari oleh pemula
  4. Data science menggunakan Python dan R sebagai bahasa pemrograman utama
  5. Model machine learning dapat digunakan untuk prediksi data


## 5. Preprocessing Teks

Lakukan preprocessing pada setiap dokumen

In [ ]:
preprocessor = TextPreprocessor()
processed_docs = []

for i in range(len(documents_text)):
    doc = documents_text[i]
    tokens = preprocessor.preprocess(doc)
    processed_docs.append(tokens)

    print(f"\nDokumen {i + 1}:")
    print(f"  Original: {doc}")
    print(f"  Tokens  : {tokens}")
    print(f"  Jumlah token: {len(tokens)}")


Dokumen 1:
  Original: Python adalah bahasa pemrograman yang populer untuk machine learning
  Tokens  : ['python', 'bahasa', 'pemrograman', 'populer', 'machine', 'learning']
  Jumlah token: 6

Dokumen 2:
  Original: Machine learning membutuhkan data yang banyak untuk pelatihan model
  Tokens  : ['machine', 'learning', 'membutuhkan', 'data', 'banyak', 'pelatihan', 'model']
  Jumlah token: 7

Dokumen 3:
  Original: Bahasa pemrograman Python mudah dipelajari oleh pemula
  Tokens  : ['bahasa', 'pemrograman', 'python', 'mudah', 'dipelajari', 'pemula']
  Jumlah token: 6

Dokumen 4:
  Original: Data science menggunakan Python dan R sebagai bahasa pemrograman utama
  Tokens  : ['data', 'science', 'menggunakan', 'python', 'bahasa', 'pemrograman', 'utama']
  Jumlah token: 7

Dokumen 5:
  Original: Model machine learning dapat digunakan untuk prediksi data
  Tokens  : ['model', 'machine', 'learning', 'digunakan', 'prediksi', 'data']
  Jumlah token: 6


## 6. Ekstraksi Fitur TF-IDF

Hitung TF-IDF untuk setiap dokumen

In [ ]:
tfidf_extractor = TFIDFExtractor()
tfidf_vectors = tfidf_extractor.fit_transform(processed_docs)

print(f"Jumlah fitur (vocabulary): {len(tfidf_extractor.get_feature_names())}")

Jumlah fitur (vocabulary): 19


## 7. Vocabulary (Fitur)

Tampilkan semua kata unik yang menjadi fitur

In [ ]:
vocabulary = tfidf_extractor.get_feature_names()
print(f"\nJumlah fitur unik: {len(vocabulary)}")
print(f"\nDaftar fitur: {vocabulary}")


Jumlah fitur unik: 19

Daftar fitur: ['bahasa', 'banyak', 'data', 'digunakan', 'dipelajari', 'learning', 'machine', 'membutuhkan', 'menggunakan', 'model', 'mudah', 'pelatihan', 'pemrograman', 'pemula', 'populer', 'prediksi', 'python', 'science', 'utama']


## 8. Nilai IDF

Tampilkan nilai IDF untuk setiap kata dalam vocabulary

In [ ]:
print("\nKata                  IDF")
print("-" * 35)
for word in vocabulary:
    idf_val = tfidf_extractor.idf_values[word]
    padding = " " * (20 - len(word))
    print(f"{word}{padding}: {format_angka(idf_val)}")


Kata                  IDF
-----------------------------------
bahasa              : 0.5108
banyak              : 1.6094
data                : 0.5108
digunakan           : 1.6094
dipelajari          : 1.6094
learning            : 0.5108
machine             : 0.5108
membutuhkan         : 1.6094
menggunakan         : 1.6094
model               : 0.9163
mudah               : 1.6094
pelatihan           : 1.6094
pemrograman         : 0.5108
pemula              : 1.6094
populer             : 1.6094
prediksi            : 1.6094
python              : 0.5108
science             : 1.6094
utama               : 1.6094


Semakin tinggi nilai IDF, semakin jarang kata tersebut muncul di dokumen

## 9. Vektor TF-IDF untuk Setiap Dokumen

Tampilkan nilai TF-IDF tertinggi untuk setiap dokumen

In [ ]:
for i in range(len(tfidf_vectors)):
    tfidf_vec = tfidf_vectors[i]
    doc_tokens = processed_docs[i]

    # Gabungkan tokens menjadi string
    doc_str = ""
    for token in doc_tokens:
        doc_str += token + " "

    print(f"\nDokumen {i + 1}: {doc_str}")
    cetak_garis(70, "-")

    # Urutkan kata berdasarkan nilai TF-IDF
    sorted_tfidf = tfidf_extractor.urutkan_tfidf(tfidf_vec)

    # Tampilkan kata dengan TF-IDF > 0
    print("Kata (TF-IDF > 0):")
    print("\nKata                  TF-IDF")
    print("-" * 35)
    count = 0
    for item in sorted_tfidf:
        word = item[0]
        score = item[1]
        if score > 0:
            padding = " " * (20 - len(word))
            print(f"  {word}{padding}: {format_angka(score, 6)}")
            count += 1

    if count == 0:
        print("  (Tidak ada kata dengan TF-IDF > 0)")


Dokumen 1: python bahasa pemrograman populer machine learning 
----------------------------------------------------------------------
Kata (TF-IDF > 0):

Kata                  TF-IDF
-----------------------------------
  populer             : 0.268240
  bahasa              : 0.085138
  learning            : 0.085138
  machine             : 0.085138
  pemrograman         : 0.085138
  python              : 0.085138

Dokumen 2: machine learning membutuhkan data banyak pelatihan model 
----------------------------------------------------------------------
Kata (TF-IDF > 0):

Kata                  TF-IDF
-----------------------------------
  banyak              : 0.229920
  membutuhkan         : 0.229920
  pelatihan           : 0.229920
  model               : 0.130899
  data                : 0.072975
  learning            : 0.072975
  machine             : 0.072975

Dokumen 3: bahasa pemrograman python mudah dipelajari pemula 
--------------------------------------------------------------

## 10. Representasi Matriks TF-IDF

Tampilkan TF-IDF dalam bentuk matriks (dokumen × fitur)

In [ ]:
print(f"\nDimensi: {len(tfidf_vectors)} dokumen × {len(vocabulary)} fitur\n")

# Tampilkan 5 fitur pertama sebagai contoh
num_features_to_show = min(5, len(vocabulary))
print(f"Menampilkan {num_features_to_show} fitur pertama:\n")

# Header
print("Dok  ", end="")
for j in range(num_features_to_show):
    word = vocabulary[j]
    padding_left = (10 - len(word)) // 2
    padding_right = 10 - len(word) - padding_left
    print(" " * padding_left + word + " " * padding_right, end="")
print("...")

# Separator
print("-" * (5 + num_features_to_show * 10 + 3))

# Data
for i in range(len(tfidf_vectors)):
    tfidf_vec = tfidf_vectors[i]
    print(f" {i + 1}   ", end="")
    vec_list = tfidf_extractor.vector_to_list(tfidf_vec)
    for j in range(num_features_to_show):
        val = vec_list[j]
        val_str = format_angka(val, 4)
        padding_left = (10 - len(val_str)) // 2
        padding_right = 10 - len(val_str) - padding_left
        print(" " * padding_left + val_str + " " * padding_right, end="")
    print("...")


Dimensi: 5 dokumen × 19 fitur

Menampilkan 5 fitur pertama:

Dok    bahasa    banyak     data   digunakan dipelajari...
----------------------------------------------------------
 1     0.0851    0.0000    0.0000    0.0000    0.0000  ...
 2     0.0000    0.2299    0.0730    0.0000    0.0000  ...
 3     0.0851    0.0000    0.0000    0.0000    0.2682  ...
 4     0.0730    0.0000    0.0730    0.0000    0.0000  ...
 5     0.0000    0.0000    0.0851    0.2682    0.0000  ...


## 11. Statistik dan Analisis

In [ ]:
# Hitung statistik
total_tokens = 0
for doc in processed_docs:
    total_tokens += len(doc)

avg_tokens = total_tokens / len(processed_docs) if len(processed_docs) > 0 else 0

print(f"  Statistik Umum:")
print(f"  • Jumlah dokumen        : {len(documents_text)}")
print(f"  • Total token (setelah preprocessing): {total_tokens}")
print(f"  • Rata-rata token per dokumen: {format_angka(avg_tokens, 2)}")
print(f"  • Jumlah fitur unik     : {len(vocabulary)}")

# Cari kata dengan IDF tertinggi dan terendah
max_idf = -999999
min_idf = 999999
max_word = ""
min_word = ""

for word in vocabulary:
    idf = tfidf_extractor.idf_values[word]
    if idf > max_idf:
        max_idf = idf
        max_word = word
    if idf < min_idf:
        min_idf = idf
        min_word = word

print(f"\n  Kata dengan IDF tertinggi:")
print(f"  • '{max_word}' dengan IDF = {format_angka(max_idf, 4)}")

print(f"\n  Kata dengan IDF terendah:")
print(f"  • '{min_word}' dengan IDF = {format_angka(min_idf, 4)}")

  Statistik Umum:
  • Jumlah dokumen        : 5
  • Total token (setelah preprocessing): 32
  • Rata-rata token per dokumen: 6.40
  • Jumlah fitur unik     : 19

  Kata dengan IDF tertinggi:
  • 'banyak' dengan IDF = 1.6094

  Kata dengan IDF terendah:
  • 'bahasa' dengan IDF = 0.5108


## 12. Menggunakan dokumen lain

In [ ]:
# Ganti dengan dokumen lain
dokumen_baru = [
    "Artificial intelligence mengubah cara kita bekerja",
    "Deep learning adalah subset dari machine learning",
    "Natural language processing membantu komputer memahami bahasa manusia"
]

# Preprocessing
preprocessor_baru = TextPreprocessor()
docs_terproses = []

print("\n1. Preprocessing:")
for i, doc in enumerate(dokumen_baru, 1):
    tokens = preprocessor_baru.preprocess(doc)
    docs_terproses.append(tokens)
    print(f"  Dok {i}: {tokens}")

# TF-IDF
tfidf_baru = TFIDFExtractor()
vectors_baru = tfidf_baru.fit_transform(docs_terproses)

print("\n2. Vocabulary:")
print(f"  {tfidf_baru.get_feature_names()}")

print("\n3. Top kata per dokumen:")
for i, vec in enumerate(vectors_baru, 1):
    sorted_vec = tfidf_baru.urutkan_tfidf(vec)
    print(f"\n  Dokumen {i}:")
    for word, score in sorted_vec[:3]:  # Top 3
        if score > 0:
            print(f"    - {word}: {format_angka(score, 4)}")


1. Preprocessing:
  Dok 1: ['artificial', 'intelligence', 'mengubah', 'cara', 'kita', 'bekerja']
  Dok 2: ['deep', 'learning', 'subset', 'machine', 'learning']
  Dok 3: ['natural', 'language', 'processing', 'membantu', 'komputer', 'memahami', 'bahasa', 'manusia']

2. Vocabulary:
  ['artificial', 'bahasa', 'bekerja', 'cara', 'deep', 'intelligence', 'kita', 'komputer', 'language', 'learning', 'machine', 'manusia', 'memahami', 'membantu', 'mengubah', 'natural', 'processing', 'subset']

3. Top kata per dokumen:

  Dokumen 1:
    - artificial: 0.1831
    - bekerja: 0.1831
    - cara: 0.1831

  Dokumen 2:
    - learning: 0.4394
    - deep: 0.2197
    - machine: 0.2197

  Dokumen 3:
    - bahasa: 0.1373
    - komputer: 0.1373
    - language: 0.1373


## Kesimpulan

### Yang Telah Dipelajari:
1. Preprocessing teks (lowercase, remove punctuation, tokenization, stopwords removal)
2. Perhitungan Term Frequency (TF)
3. Perhitungan Inverse Document Frequency (IDF)
4. Perhitungan TF-IDF
5. Representasi dokumen dalam bentuk vektor numerik

### Keuntungan Membuat from Scratch Tanpa Library:
- Memahami konsep fundamental
- Dapat melakukan customisasi sesuai kebutuhan
- Dasar yang kuat untuk mempelajari teknik NLP lanjutan